In [1]:
import sympy as sm
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import timeit

In [4]:
def RCP (t, u, r1, q1, aC1, eC1, hC1, mC1, aP1, eP1, mP1, hP1): 
    
    du = np.zeros([4, 1])
    
    #variables
    R = u[0]
    C = u[1]
    P = u[2]

    
    du[0] = r1*R*(1 - (R*q1)) - (aC1*R*C)/(1 + (aC1*hC1*R))
    du[1] = (eC1*aC1*R*C)/(1 + (aC1*hC1*R)) - (aP1*C*P)/(1+ (aP1*hP1*C)) - mC1*C
    du[2] = (eP1*aP1*C*P)/(1+ (aP1*hP1*C)) - mP1*P
    du[3] = 1
    
    du = du.reshape(4,)
    return(du)

In [5]:
pars = pd.read_csv('param_hycub_H2.csv')
par1 = np.array(pars)
np.shape(par1)

(10, 1000)

In [6]:
i=3
r0 = par1[0][i]; q0 =  par1[1][i]
aC0 = par1[2][i]; eC0 = par1[3][i]; mC0 = par1[4][i]; hC0 = par1[5][i]
aP0 = par1[6][i]; eP0 = par1[7][i]; mP0 = par1[8][i]; hP0 = par1[9][i]
p = (r0, q0, aC0, eC0, hC0, mC0, aP0, eP0, mP0, hP0)
print(p)

(3.643600021, 3.15e-05, 0.386839673, 0.184338712, 0.041651292, 0.340858499, 0.020505594, 0.269601636, 0.573379826, 0.047695049)


In [7]:
def oscillation_profile (jump, mv, df): #jump = time steps for oscillation profiling, mv = moving window, df = data
    
    import scipy.signal as sig; import numpy as np; 
    import pandas as pd; import matplotlib.pyplot as plt
    
    df.columns = ['time', 'sv'] ##rename the columns
    
    ln = np.shape(df)[0] ##get the length of data to calculate the frequency & amplitude
    ids = (np.around(np.linspace(0,ln,int(ln/jump)),0)) ##the start point for iterations

    ncyc = np.zeros(10000); amp_mn = np.zeros(10000); ##initiate a numpy array to store info about cycles & amplitude

    for i in np.arange(len(ids)): #iterations for the number of start points
        rows = ids[i]   ##start row number
        rowe = mv + ids[i] ##end row number
        
    
        if rowe <= ln: ##if the rows ids are within the range of data
            iter = i  ##get the iteration information 
            dfi = df.loc[rows:rowe]  ##subset the data
            peaks, _ = sig.find_peaks(dfi['sv'], height=1) ##get the maxima information
            ncyc[i] = len(peaks)  ##number of cycles  = number of peaks in the time series
            amp_mn[i] = np.mean(dfi.sv.values[peaks])  ## mean of the amplitude of the oscillations
            
#     tm = np.arange(iter)
#     plt.plot(tm, amp_mn[:iter])
    
    mvv = dfi.time.values[dfi.shape[0]-1] - dfi.time.values[0] ##get the actual time difference from data
            
    ncyc1 = ncyc[:iter]  ##Get the cycles information till the last interation conducted
    amp_mn1 = amp_mn[:iter]
    freq = np.mean(ncyc1)/mvv ## frequency  = number of cycles / time period (moving window)
    ampl_mn = np.mean(amp_mn1) ##mean of the amplidtude
    ampl_sd = np.sqrt(np.var(amp_mn1)) ##variance in the amplitude
    
    res = np.array([freq, ampl_mn, ampl_sd]) #collect the results
    
    return(res)     

In [3]:
import lhsmdu as lh
prs = lh.createRandomStandardUniformMatrix(100,4)
pr1 = prs
pr1[:,0] = pr1[:,0] * 10000 + 2000
pr1[:,1] = pr1[:,1] * 5000 + 500
pr1[:,2] = pr1[:,2] * 1000 + 10
pr1[:,3] = pr1[:,3] * 0

In [4]:
pr1[1,:]

array([3560.18640442, 1279.97260168,   68.08361217,    0.        ])

In [11]:
from joblib import Parallel, delayed

def stat_RCP_process(i):

    u0 = pr1[i,:]
    tend = 10000
    t_span = (0, tend)
    tint = np.linspace(0, tend, tend*10)

    sol1 = solve_ivp (RCP, t_span, u0, args = p, t_eval = tint, method= 'RK45', rtol=1e-06, atol=1e-06)
    print([i, 'simu over'])

    ssv0 = sol1.y[:,5000:]
    tm = ssv0[3]; R0 = ssv0[0]; C0 = ssv0[1]; P0 = ssv0[2]

    Rdf = np.array([tm, R0])
    Rdf = pd.DataFrame(np.transpose(Rdf))
    resource_df = oscillation_profile(100, 5000, Rdf)

    Cdf = np.array([tm, C0])
    Cdf = pd.DataFrame(np.transpose(Cdf))
    consumer_df = oscillation_profile(100, 5000, Cdf)

    Pdf = np.array([tm, P0])
    Pdf = pd.DataFrame(np.transpose(Pdf))
    predator_df = oscillation_profile(100, 5000, Pdf)

    rs = np.column_stack((resource_df,consumer_df,predator_df))
    return(rs)

In [2]:
#Parallel processing has been conducted using the following example#
from joblib import Parallel, delayed
def process(i):
    return i * i
    
results = Parallel(n_jobs=2)(delayed(process)(i) for i in range(10))
print(results)  # prints [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [12]:
results = Parallel(n_jobs=4)(delayed(stat_RCP_process)(i) for i in np.arange(np.shape(prs)[0]))
results

[array([[5.74365780e-02, 5.74165560e-02, 5.74343533e-02],
        [3.17459400e+04, 7.44076085e+03, 1.06027340e+03],
        [5.97089271e-04, 1.28873569e+01, 1.28868747e+00]]),
 array([[5.74187806e-02, 5.74210053e-02, 5.74187806e-02],
        [3.17459400e+04, 7.44390594e+03, 1.06031660e+03],
        [9.91008608e-04, 1.36336641e+01, 1.59287161e+00]]),
 array([[5.74343533e-02, 5.74321287e-02, 5.74365780e-02],
        [3.17459399e+04, 7.44383220e+03, 1.06039434e+03],
        [9.26628676e-04, 1.03728097e+01, 1.76148037e+00]]),
 array([[5.74299040e-02, 5.74321287e-02, 5.74210053e-02],
        [3.17459399e+04, 7.44757746e+03, 1.06036412e+03],
        [7.78161260e-04, 1.30947026e+01, 1.32310000e+00]]),
 array([[5.74232300e-02, 5.74365780e-02, 5.74165560e-02],
        [3.17459399e+04, 7.44617147e+03, 1.06043324e+03],
        [7.12779023e-04, 1.23931897e+01, 1.44498801e+00]]),
 array([[5.74365780e-02, 5.74365780e-02, 5.74365780e-02],
        [3.17459398e+04, 7.44514388e+03, 1.06044868e+03],
    

[2, 'simu over']
[9, 'simu over']
[15, 'simu over']
[21, 'simu over']
[27, 'simu over']
[30, 'simu over']
[33, 'simu over']
[37, 'simu over']
[40, 'simu over']
[43, 'simu over']
[46, 'simu over']
[49, 'simu over']
[52, 'simu over']
[55, 'simu over']
[58, 'simu over']
[61, 'simu over']
[64, 'simu over']
[68, 'simu over']
[71, 'simu over']
[75, 'simu over']
[79, 'simu over']
[82, 'simu over']
[86, 'simu over']
[89, 'simu over']
[93, 'simu over']
[96, 'simu over']
[3, 'simu over']
[5, 'simu over']
[8, 'simu over']
[11, 'simu over']
[13, 'simu over']
[16, 'simu over']
[18, 'simu over']
[22, 'simu over']
[24, 'simu over']
[28, 'simu over']
[32, 'simu over']
[36, 'simu over']
[41, 'simu over']
[47, 'simu over']
[53, 'simu over']
[59, 'simu over']
[63, 'simu over']
[67, 'simu over']
[70, 'simu over']
[74, 'simu over']
[77, 'simu over']
[81, 'simu over']
[84, 'simu over']
[88, 'simu over']
[91, 'simu over']
[95, 'simu over']
[98, 'simu over']
[1, 'simu over']
[6, 'simu over']
[12, 'simu over']

In [13]:
np.shape(results)

(100, 3, 3)

In [18]:
reshaped_array=np.reshape(results,(100,9))
np.shape(reshaped_array)

(100, 9)

In [22]:
iter3_ICs_osc = pd.DataFrame(reshaped_array)
iter3_ICs_osc.columns = ['R_fr', 'C_fr', 'P_fr', 'R_amp', 'C_amp', 'P_amp', 'R_vamp', 'C_vamp', 'P_vamp']
iter3_ICs_osc.to_csv('iter3_ICs_oscillation_profile.csv')

In [2]:
i3sdf = pd.read_csv("iter3_ICs_oscillation_profile.csv")
i3sdf = i3sdf.iloc[: , 1:]
i3sdf

,R_fr,C_fr,P_fr,R_amp,C_amp,P_amp,R_vamp,C_vamp,P_vamp
0,0.057437,0.057417,0.057434,31745.940031,7440.760853,1060.273401,0.000597,12.887357,1.288687
1,0.057419,0.057421,0.057419,31745.940045,7443.905939,1060.316595,0.000991,13.633664,1.592872
2,0.057434,0.057432,0.057437,31745.939926,7443.832203,1060.394337,0.000927,10.372810,1.761480
3,0.057430,0.057432,0.057421,31745.939899,7447.577457,1060.364120,0.000778,13.094703,1.323100
4,0.057423,0.057437,0.057417,31745.939934,7446.171466,1060.433244,0.000713,12.393190,1.444988
...,...,...,...,...,...,...,...,...,...
95,0.057397,0.057397,0.057410,31745.940172,7444.192766,1059.732443,0.000754,10.586244,1.474234
96,0.057461,0.057470,0.057472,31745.939587,7442.162153,1061.072065,0.000836,13.184853,1.498363
97,0.057441,0.057450,0.057452,31745.939829,7442.402664,1060.510810,0.000709,9.417395,1.406036
98,0.057388,0.057401,0.057394,31745.940263,7443.528618,1059.967247,0.000984,10.998608,1.580785
